In [1]:
import json

In [2]:
with open('train-test.json') as fopen:
    train_test = json.load(fopen)
    
train_test.keys()

dict_keys(['train_X', 'test_X', 'train_Y', 'test_Y'])

In [3]:
with open('combined.txt', 'w') as fopen:
    fopen.write('\n'.join(train_test['train_X'] + train_test['test_X']))

In [4]:
import youtokentome as yttm

In [5]:
%%time

bpe = yttm.BPE.train(data='combined.txt', 
               vocab_size=300000, model='language-detection.model')

CPU times: user 2min 1s, sys: 24.5 s, total: 2min 25s
Wall time: 34.3 s


In [6]:
vocab = {v: i for i, v in enumerate(bpe.vocab())}
rev_vocab = {i: v for i, v in enumerate(bpe.vocab())}
len(vocab)

300000

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
import re

r = re.compile(r'[\S]+').findall

In [9]:
combined = train_test['train_X'] + train_test['test_X']

In [10]:
subs = [' '.join(s) for s in bpe.encode(combined[:3], output_type=yttm.OutputType.SUBWORD)]
subs[0]

'▁iyaa ▁kalo ▁dibandingin ▁sm ▁my ▁pendukungnya ▁najib ▁ya ▁yg ▁model ▁bgini ▁ini ▁kesel ▁ya ▁beneran'

In [11]:
subs = [' '.join(s) for s in bpe.encode(combined, output_type=yttm.OutputType.SUBWORD)]

In [12]:
len(subs)

11242015

In [13]:
bow = CountVectorizer(vocabulary = vocab, token_pattern = r'[\S]+').fit(subs)

In [14]:
subs[1]

'▁aku ▁bodoh ▁tak ▁macam ▁adik ▁beradik ▁aku ▁yang ▁lain'

In [15]:
import numpy as np

v = np.array(bow.transform(subs[1:2]).todense())[0]
np.where(v > 0), v[v > 0]

((array([ 7521,  7631,  7634,  7802,  8310,  9659, 10726, 17074]),),
 array([1, 1, 2, 1, 1, 1, 1, 1]))

In [16]:
import pickle
with open('bow-language-detection.pkl','wb') as fopen:
    pickle.dump(bow,fopen)